# Generate results for eval section in paper

In [ ]:
import json
import shutil
import sys
from pathlib import Path
import numpy as np
sys.path.append("/home/pauldelseith/HyperDiffusion_semantic_decomposition")
import torch
import trimesh
import logging

In [ ]:
# Check if CUDA is available
device = "cpu"
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")

In [ ]:
import evaluation_metrics_3d
# Shape novelty
n_points = 2048
gen_meshes_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/gen_meshes/serene-fog")
ref_pcs_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/sem_seg_meshes/Chair_reduced_2048_pc")
train_shape_ids = list(np.genfromtxt(ref_pcs_dir / "train_split.lst", dtype="str"))
# load ground truth train pcs
pcs = []
for shape_id in train_shape_ids:
    pc = np.load(ref_pcs_dir / f"{shape_id}.npy")
    pc = torch.tensor(pc).float()
    pcs.append(pc)
ref_pcs = torch.stack(pcs)
pcs = []
gen_files = []
for mesh_file in gen_meshes_dir.glob("*.obj"):
    mesh = trimesh.load(mesh_file)
    if len(mesh.vertices) == 0:
        continue
    gen_files.append(mesh_file.name)
    mesh = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
    pc = torch.tensor(mesh.sample(n_points))
    pc = pc.float()
    pcs.append(pc)
gen_pcs = torch.stack(pcs)
batch_size = gen_pcs.shape[0]

In [ ]:
k=3
ref_pcs = ref_pcs.to(device)
gen_pcs = gen_pcs.to(device)
batch_size = torch.tensor(batch_size).to(device)

M_rs_cd, M_rs_emd = evaluation_metrics_3d._pairwise_EMD_CD_(ref_pcs, gen_pcs, batch_size)
M_rr_cd, M_rr_emd = evaluation_metrics_3d._pairwise_EMD_CD_(ref_pcs, ref_pcs, batch_size)
M_ss_cd, M_ss_emd = evaluation_metrics_3d._pairwise_EMD_CD_(gen_pcs, gen_pcs, batch_size)

# get k nearest neighbors
val, idx = M_rs_cd.topk(k, dim=0, largest=False)

In [ ]:
for i, mesh_name in enumerate(gen_files):
    print(f"Mesh {mesh_name}")
    print(f"Nearest neighbors:")
    for j in range(k):
        print(f"  {mesh_name} to {train_shape_ids[idx[j, i]]} with distance {val[j, i]}")
    print("")